In [24]:
import os

In [25]:
%pwd

'd:\\MachineLearning\\ml\\projectmlops\\mlopsproject\\research'

In [26]:
os.chdir("../")

In [27]:
%pwd

'd:\\MachineLearning\\ml\\projectmlops\\mlopsproject'

In [28]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [29]:
#configuration

from mlops.constants import *
from mlops.utils.common import read_yaml, create_directories

In [30]:
# Configuration


class ConfigurationManager:
        def __init__(
                self, 
                config_filepath = CONFIG_FILE_PATH, 
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):

                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)

                create_directories([self.config.artifacts_root])
        

        def get_data_ingestion_config(self) -> DataIngestionConfig:
                config = self.config.data_ingestion

                create_directories([config.root_dir])

                data_ingestion_config = DataIngestionConfig(
                        root_dir = config.root_dir, 
                        source_URL = config.source_URL,
                        local_data_file = config.local_data_file,
                        unzip_dir = config.unzip_dir
                )
                return data_ingestion_config




In [31]:
#components_data-ingestion

import os
import urllib.request as request
import zipfile
from mlops import logger
from mlops.utils.common import get_size



In [35]:
#components_data_ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with the following info: \n{headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

        
    def extract_zip_file(self):
        """
        zip_file_path: str
        extracts the zip file in the data directory
        fucntion returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [36]:
#pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-24 18:04:46,776: INFO: common: yaml file:config\config.yaml loaded sucessfully]
[2023-09-24 18:04:46,778: INFO: common: yaml file:params.yaml loaded sucessfully]
[2023-09-24 18:04:46,782: INFO: common: yaml file:schema.yaml loaded sucessfully]
[2023-09-24 18:04:46,784: INFO: common: Created directory at : artifacts]
[2023-09-24 18:04:46,785: INFO: common: Created directory at : artifacts/data_ingestion]
[2023-09-24 18:04:48,820: INFO: 898812774: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 26164
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "aa44bdcb20d3f855c129123828f3c2092ec23e3aeec7dab32644290cf31c0ede"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FEF0:45E0:2FEB8C:3C2A0E:651025E2
Accept-Ranges: bytes
Date: Sun, 24 Se